In [2]:
# Imports

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
# Obtain URL and get request:

url = "https://www.coingecko.com/"

response = requests.get(url)

# Check Status code:

response.status_code

200

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
results = soup.find('tbody').find_all('tr')

## Obtain Data

In [6]:
# Rank Numbers
ranks = [x.find('td', {'class': "table-number tw-text-left text-xs cg-sticky-col cg-sticky-second-col tw-max-w-14 lg:tw-w-14"}).text.strip()
         for x in results]

In [7]:
# Crypto Names
names = [x.find('a', {'class': "tw-hidden lg:tw-flex font-bold tw-items-center tw-justify-between"}).text.strip()
         for x in results]

In [8]:
# Crypto Tickers:
tickers = [x.find('a', {'class': "d-lg-none font-bold tw-w-12"}).text.strip() for x in results]

In [9]:
# Prices (per Unit)

prices = [x.find('span', {'class': "no-wrap"}).text.strip() for x in results]

In [10]:
# % Change 1h

pct_change_1h = [x.find('td', {'class': "td-change1h change1h stat-percent text-right col-market"}).text.strip() 
                 for x in results]

In [11]:
# % Change 24h

pct_change_24h = [x.find('td', {'class': "td-change24h change24h stat-percent text-right col-market"}).text.strip() 
                  for x in results]

In [12]:
# % Change 7d

pct_change_7d = [x.find('td', {'class': "td-change7d change7d stat-percent text-right col-market"}).text.strip() 
                 for x in results]

In [13]:
# 24hr volume
volume_24h = [x.find('td', {'class': "td-liquidity_score lit text-right col-market"}).text.strip() 
              for x in results]

In [14]:
# Market Cap
market_cap = [x.find('td', {'class': "td-market_cap cap col-market cap-price text-right"}).text.strip() 
              for x in results]

In [15]:
## Create dataframe:

coingecko_top100_df = pd.DataFrame({
                        'Rank': ranks,
                        'Crypto': names,
                        'Ticker': tickers,
                        'Price': prices,
                        'Pct_Change_1h': pct_change_1h,
                        'Pct_Change_24h': pct_change_24h,
                        'Pct_Change_7d': pct_change_7d,
                        'Volume (24h)': volume_24h,
                        'Market Cap': market_cap
})

In [16]:
# Preview:
coingecko_top100_df.head(10)

,Rank,Crypto,Ticker,Price,Pct_Change_1h,Pct_Change_24h,Pct_Change_7d,Volume (24h),Market Cap
0,1,Bitcoin,BTC,"$38,864.74",-0.8%,7.8%,-4.2%,"$28,275,028,167","$737,479,703,294"
1,2,Ethereum,ETH,"$2,689.27",-1.0%,9.1%,-6.7%,"$20,377,671,989","$322,064,170,813"
2,3,Tether,USDT,$1.00,0.0%,0.0%,0.1%,"$59,469,440,560","$79,662,600,445"
3,4,BNB,BNB,$366.47,-0.8%,5.8%,-9.0%,"$1,645,977,779","$61,636,998,238"
4,5,USD Coin,USDC,$1.00,-0.2%,-0.1%,0.1%,"$5,387,680,572","$53,233,835,623"
5,6,XRP,XRP,$0.708108,-0.4%,7.0%,-7.9%,"$4,560,690,658","$33,969,741,813"
6,7,Solana,SOL,$89.85,-1.0%,5.8%,-3.8%,"$2,737,158,288","$28,747,152,864"
7,8,Cardano,ADA,$0.874336,-0.8%,7.7%,-14.4%,"$1,577,141,276","$28,038,551,037"
8,9,Terra,LUNA,$69.54,1.0%,19.3%,38.1%,"$3,582,844,605","$26,486,680,783"
9,10,Avalanche,AVAX,$77.64,-0.2%,10.1%,-12.0%,"$1,731,796,384","$19,068,342,123"


In [ ]:
# Write to excel file:
from datetime import date

coingecko_top100_df.to_excel("Coingecko_Top100" + str(date.today()) + ".xlsx", index = False)

In [ ]:
# Save to .csv:

coingecko_top100_df.to_csv("Coingecko_Top100" + str(date.today()) + ".csv", index = False)

## Multiple Pages Case

In [18]:
# Testing f string with for loop
for i in range(1, 4):
   test = f"https://www.coingecko.com/?page={i}"
   print(test)

https://www.coingecko.com/?page=1
https://www.coingecko.com/?page=2
https://www.coingecko.com/?page=3


In [34]:
# Loop through each page, extract information for each crypto:
# Each page has 100 cryptos
# Reference: https://www.youtube.com/watch?v=JbXF3UaaqC8

num_pages = 5

# Initalize lists:

ranks_list = []
names_list = []
tickers_list = []
price_list = []
pct_change_1h_list = []
pct_change_24h_list = []
pct_change_7d_list = []
volume_24h_list = []
market_cap_list = []

for i in range(1, num_pages + 1):
    # Obtain URL and get request:
    url = f"https://www.coingecko.com/?page={i}"
    response = requests.get(url)
    
    # Soup & results:
    soup = BeautifulSoup(response.content, 'html.parser')
    results = soup.find('table', {'class': 'table-scrollable'}).find('tbody').find_all('tr')
    
    for result in results:
        # Rank:
        try:
            ranks_list.append(result.find('td', 
                              {'class': "table-number tw-text-left text-xs cg-sticky-col cg-sticky-second-col tw-max-w-14 lg:tw-w-14"}).text.strip())
        except:
            ranks_list.append("N/A")
    
        # Crypto Name:
        try:
            names_list.append(result.find('a', 
                             {'class': "tw-hidden lg:tw-flex font-bold tw-items-center tw-justify-between"}).text.strip())                  
        except:
            names_list.append("N/A")
    
        # Tickers:
        try:
            tickers_list.append(result.find('a', {'class': "d-lg-none font-bold tw-w-12"}).text.strip())                  
        except:
            tickers_list.append("N/A")
    
        # Price:
        try:
            price_list.append(result.find('span', {'class': "no-wrap"}).text.strip())        
        except:
            price_list.append("N/A")                  
                              
        # % Change 1h
        try:
            pct_change_1h_list.append(result.find('td', 
                                    {'class': "td-change1h change1h stat-percent text-right col-market"}).text.strip())                
        except:
            pct_change_1h_list.append("N/A")
    
        # % Change 24h
        try:
            pct_change_24h_list.append(result.find('td', 
                                      {'class': "td-change24h change24h stat-percent text-right col-market"}).text.strip())                              
        except:
            pct_change_24h_list.append("N/A")
    
        # % Change 7d                                      
        try:
            pct_change_7d_list.append(result.find('td', 
                                     {'class': "td-change7d change7d stat-percent text-right col-market"}).text.strip())                              
        except:
            pct_change_7d_list.append("N/A")
    
        # Volume 24h
        try:
            volume_24h_list.append(result.find('td', 
                                  {'class': "td-liquidity_score lit text-right col-market"}).text.strip())                              
        except:
            volume_24h_list.append("N/A")
    
        # Market Cap:
        try:
            market_cap_list.append(result.find('td', {'class': "td-market_cap cap col-market cap-price text-right"}).text.strip())                              
        except:
            market_cap_list.append("N/A")
    

In [36]:
# Create dataframe of multiple pages of Coingecko crypto price data:

coingecko_df = pd.DataFrame({
                        'Rank': ranks_list,
                        'Crypto': names_list,
                        'Ticker': tickers_list,
                        'Price': price_list,
                        'Pct_Change_1h': pct_change_1h_list,
                        'Pct_Change_24h': pct_change_24h_list,
                        'Pct_Change_7d': pct_change_7d_list,
                        'Volume (24h)': volume_24h_list,
                        'Market Cap': market_cap_list
})

In [39]:
coingecko_df.head(10)

,Rank,Crypto,Ticker,Price,Pct_Change_1h,Pct_Change_24h,Pct_Change_7d,Volume (24h),Market Cap
0,1,Bitcoin,BTC,"$38,702.88",-1.0%,6.3%,-4.6%,"$27,656,764,794","$733,816,688,308"
1,2,Ethereum,ETH,"$2,678.14",-0.9%,6.3%,-7.1%,"$19,805,092,705","$320,646,854,786"
2,3,Tether,USDT,$1.00,-0.1%,-0.0%,-0.1%,"$58,558,167,629","$79,616,949,952"
3,4,BNB,BNB,$365.10,-0.9%,3.9%,-9.3%,"$1,610,209,739","$61,387,323,881"
4,5,USD Coin,USDC,$0.999559,0.1%,0.1%,0.0%,"$5,178,000,688","$53,243,188,399"
5,6,XRP,XRP,$0.709652,0.3%,5.6%,-7.7%,"$4,545,734,779","$34,006,470,717"
6,7,Solana,SOL,$89.45,-1.2%,3.0%,-4.2%,"$2,706,400,250","$28,749,929,019"
7,8,Cardano,ADA,$0.869926,-1.0%,4.7%,-14.8%,"$1,536,900,424","$27,895,402,917"
8,9,Terra,LUNA,$69.01,0.0%,14.7%,37.1%,"$3,585,619,403","$26,491,150,025"
9,10,Avalanche,AVAX,$77.26,-0.8%,7.2%,-12.4%,"$1,704,475,101","$19,113,157,595"


In [40]:
coingecko_df.tail(10)

,Rank,Crypto,Ticker,Price,Pct_Change_1h,Pct_Change_24h,Pct_Change_7d,Volume (24h),Market Cap
290,291,StarLink,STARL,$0.000016790919,-1.5%,17.6%,-14.6%,"$11,330,384","$169,364,717"
291,292,Bifrost,BFC,$0.158785,-0.4%,8.7%,-5.4%,"$3,416,488","$169,031,297"
292,293,aelf,ELF,$0.359869,-2.2%,27.1%,8.1%,"$227,178,762","$165,986,287"
293,294,Mirror Protocol,MIR,$1.16,-0.5%,11.4%,-14.4%,"$17,789,902","$165,975,126"
294,295,Trust Wallet Token,TWT,$0.474674,-1.0%,9.0%,-12.5%,"$4,985,694","$165,339,004"
295,296,Lido DAO,LDO,$1.62,0.5%,11.1%,-0.8%,"$3,306,933","$164,766,352"
296,297,Ion,ION,"$7,690.65",-0.7%,5.5%,-12.5%,"$131,676","$164,634,423"
297,298,AscendEx Token,ASD,$0.220522,-0.4%,1.3%,-11.8%,"$2,279,050","$163,793,476"
298,299,Request,REQ,$0.210548,0.3%,8.4%,-10.5%,"$20,319,704","$162,397,801"
299,300,Divi,DIVI,$0.059132325846,-0.2%,-0.6%,-24.2%,"$283,364","$162,226,912"


In [ ]:
# Write to excel file:
from datetime import date

coingecko_df.to_excel("Coingecko" + str(date.today()) + ".xlsx", index = False)

In [ ]:
# Save to .csv:

coingecko_df.to_csv("Coingecko_Top100" + str(date.today()) + ".csv", index = False)